<a href="https://colab.research.google.com/github/chfava/INF8953CE_Kaggle/blob/main/INF8953CE_Kaggle_(6)_(1)_(1)_(1)_(2)_(2)_(1).ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Connect Google Drive for Kaggle config file



1.  From your Kaggle account, create New API Token and download file
2. Create directory /Kaggle at the highest level in your Google Drive (My Drive/Kaggle)
3.  Place the kaggle.json file in My Drive/Kaggle/
4. Link the colab to your google drive in the following cell 



In [1]:
from google.colab import drive
drive.mount('/content/gdrive')

Mounted at /content/gdrive


In [2]:
import os
os.environ['KAGGLE_CONFIG_DIR'] = "/content/gdrive/My Drive/Kaggle"

In [3]:
PATH = 'gdrive/My Drive/Kaggle/'

In [4]:
#PATH = ''

# Download Data

In [5]:
%%capture
!kaggle competitions download -c f2020-INF8953CE

In [6]:
%%capture
!unzip -o  \*.zip

In [7]:
import numpy as np
import pandas as pd 
from matplotlib import pyplot as plt
from sklearn.model_selection import train_test_split
import tensorflow as tf
from tensorflow.keras import layers, models
from sklearn.preprocessing import OneHotEncoder
from keras.utils import to_categorical
from tensorflow.keras.callbacks import EarlyStopping
from tensorflow.keras.callbacks import ModelCheckpoint
from keras.models import load_model
from keras.layers.merge import add, concatenate, Multiply
from keras import regularizers
import cv2
from scipy.ndimage import gaussian_filter
from scipy.ndimage import map_coordinates
import json
from multiprocessing import Process
from multiprocessing import Queue

In [8]:
class Dataset:
  def __init__(self, name, preprocess = False, augment = False, deform_augment = False, alpha = [5], sigma = [2], rotate_augment = False, angles = [60], flipAND_ = False):
    self.name = name
    self.isPreprocessed = preprocess
    self.alpha = alpha
    self.sigma = sigma
    self.angles = angles
    self.IMAGE_SIZE_CROP = 32
    self.IMAGE_SIZE = 100

    self.oneHotTransformer = OneHotEncoder(handle_unknown='ignore', sparse = False)
    if preprocess == False:
      self.X_train = self.convertData(np.load('train_images.npy',allow_pickle=True, encoding = 'bytes'))
      self.X_submission = self.convertData(np.load('test_images.npy', allow_pickle=True, encoding = 'bytes'))
      self.Y_train = self.oneHotTransformer.fit_transform(pd.read_csv("train_labels.csv").Category.values.reshape(-1, 1))

    elif preprocess == True:
      self.X_train = self.convertData(self.preprocess(np.load('train_images.npy',allow_pickle=True, encoding = 'bytes'), r = int(self.IMAGE_SIZE_CROP/2)))
      self.X_submission = self.convertData(self.preprocess(np.load('test_images.npy', allow_pickle=True, encoding = 'bytes'), r = int(self.IMAGE_SIZE_CROP/2)))
      self.Y_train = self.oneHotTransformer.fit_transform(pd.read_csv("train_labels.csv").Category.values.reshape(-1, 1))

    self.splitData()

    self.prepareForFinalTrain()
    
    X_train_original = self.X_train
    Y_train_original = self.Y_train

    if augment == True:
      self.X_train = np.asarray(list(self.flip(self.X_train)) + list(self.X_train))
      self.Y_train = np.asarray(list(self.Y_train) + list(self.Y_train))

    if flipAND_ == True:
      X_train_original = self.X_train
      Y_train_original = self.Y_train

    if deform_augment == True:
      print("Augmentation of the dataset by deformation")
      Y_list = []
      q = Queue()
      p_ = []
      for i in range(0,len(self.alpha)):
        p = Process(target=self.deformation, args=(q, X_train_original, self.alpha[i], self.sigma[i], None))
        p_.append(p)
        p.start()
        print('*', end='')
        Y = np.asarray(list(Y_train_original))
        Y_list.append(Y)

      for i in range(0,len(Y_list)):
        self.X_train = np.asarray(list(self.X_train) + list(q.get()))
        self.Y_train = np.asarray( list(self.Y_train) + list(Y_list[i]))
      
      print('\n preparing for join')
      for p in p_:
        p.join()
        print('*', end='')

    print('\n')

    if rotate_augment == True:
      print("Augmentation of the dataset by rotation")
      
      Y_list = []
      q = Queue()
      p_ = []
      for angle in self.angles:
        p = Process(target=self.applyRotationToDataset, args=(X_train_original, angle, q,))
        p_.append(p)
        p.start()
        print('*', end='')
        Y = np.asarray(list(Y_train_original))
        Y_list.append(Y)
      print('\n preparing for join')
      
      for i in range(0,len(Y_list)):
        self.X_train = np.asarray(list(self.X_train) + list(q.get()))
        self.Y_train = np.asarray( list(self.Y_train) + list(Y_list[i]))

      for p in p_:
        p.join()
        print('*', end='')
        
  def convertData(self, data):

    images = []
    for dataLine in data:
      if self.isPreprocessed == False:
        images.append(np.reshape(dataLine[1], (self.IMAGE_SIZE, self.IMAGE_SIZE, 1)))
      else:
        images.append(np.reshape(dataLine, (self.IMAGE_SIZE_CROP, self.IMAGE_SIZE_CROP, 1)))
    return np.asarray(images)

  def visualizeData(self, indexBegin, indexEnd, dataset = 'X_train'):
    if dataset == 'X_train':
      data = self.X_train
    elif dataset == 'X_valid':
      data = self.X_valid
    elif dataset == 'X_test':
      data = self.X_test
    elif dataset == 'X_submission':
      data = self.X_submission
    
    for index in range(indexBegin, indexEnd):
      plt.imshow(np.reshape(data[index], (self.IMAGE_SIZE_CROP,self.IMAGE_SIZE_CROP)), cmap='Greys')
      plt.show()

  def splitData(self):
    self.X_train, self.X_test, self.Y_train, self.Y_test = train_test_split(self.X_train, self.Y_train, test_size=0.10, random_state=42)
    self.X_train, self.X_valid, self.Y_train, self.Y_valid = train_test_split(self.X_train, self.Y_train, test_size=0.20, random_state=42)
  
  def pre2(self, img, blur= True, dilate= False, min_area=70):
    blackAndWhite = img
    # Blur filter
    if blur:
      blackAndWhite = cv2.medianBlur(img, 3)

    # Find connectected components
    nlabels, labels, stats, centroids = cv2.connectedComponentsWithStats(blackAndWhite, 8, cv2.CV_8U)
    sizes = stats[1:, -1] #get CC_STAT_AREA component
    img2 = np.zeros((labels.shape), np.uint8)

    for i in range(0, nlabels - 1):
        if sizes[i] >= min_area:   #filter small dotted regions
            img2[labels == i + 1] = 255

    dilation = img2
    # Dilation of the mask
    if dilate:
      kernel = np.ones((5,5),np.uint8)
      dilation = cv2.dilate(img2, kernel, iterations = 1) 
    
    # Mask kept regions
    res = cv2.bitwise_and(img, dilation)
    return res

  def crop_img(self, img, r=15):
    nlabels, labels, stats, centroids = cv2.connectedComponentsWithStats(img, 8, cv2.CV_8U)
    tmp = []
    for c in centroids:
      x= int(round(c[0]))
      y= int(round(c[1]))

      start_x = x-r
      end_x = x+r
      if start_x < 0:
        end_x = end_x - start_x
        start_x = 0
      elif end_x > 99:
        start_x = 99 - 2*r
        end_x = 99
      
      start_y = y-r
      end_y = y+r
      if start_y < 0:
        end_y = end_y - start_y
        start_y = 0
      elif end_y > 99:
        start_y = 99 - 2*r
        end_y = 99

      tmp.append(img[start_y:end_y, start_x:end_x])

    return tmp[np.argmax([im.sum() for im in tmp])]

  # Apply preprocess
  def preprocess(self, dataset, r=15):
    dataset = dataset[:,1]
    images = []
    for img in dataset:
      temp = img.astype(np.uint8).reshape(self.IMAGE_SIZE,self.IMAGE_SIZE)
      
      res = self.pre2(temp)
      res = self.pre2(res, blur=False)
      res = self.crop_img(res, r)
        
      images.append(res.reshape(4*r*r,))
      images_np = np.array(images)

    return images_np

  def flip(self, dataset):
    dataset_flip = []
    for img in dataset:
      img = np.reshape(img, (self.IMAGE_SIZE_CROP, self.IMAGE_SIZE_CROP))
      d_flip = cv2.flip(img, 1)
      dataset_flip.append(np.reshape(d_flip, (self.IMAGE_SIZE_CROP, self.IMAGE_SIZE_CROP, 1)))

    return np.asarray(dataset_flip)

  def deformation(self, q, dataset, alpha = 5, sigma = 2, random_state=None ):
    deform_dataset = []
    for img in dataset:
      img = np.reshape(img, (self.IMAGE_SIZE_CROP, self.IMAGE_SIZE_CROP))
      shape = img.shape

      if random_state is None:
          random_state = np.random.RandomState(None)

      dx = gaussian_filter((random_state.rand(*shape) * 2 - 1), sigma, mode="constant", cval=0) * alpha
      dy = gaussian_filter((random_state.rand(*shape) * 2 - 1), sigma, mode="constant", cval=0) * alpha

      x, y = np.meshgrid(np.arange(shape[0]), np.arange(shape[1]), indexing='ij')
      indices = np.reshape(x+dx, (-1, 1)), np.reshape(y+dy, (-1, 1))
      img_deform = map_coordinates(img, indices, order=1).reshape(shape)
      img_deform = np.reshape(img_deform, (self.IMAGE_SIZE_CROP, self.IMAGE_SIZE_CROP, 1))
      deform_dataset.append(img_deform)
    
    q.put(np.asarray(deform_dataset))
  
  def applyRotationToDataset(self, dataset, angle, q):
    rotated_dataset = []
    for img in dataset:
      img = np.reshape(img, (self.IMAGE_SIZE_CROP, self.IMAGE_SIZE_CROP))
      img = self.rotate_image(img, angle)
      rotated_dataset.append(np.reshape(img, (self.IMAGE_SIZE_CROP, self.IMAGE_SIZE_CROP, 1)))
    q.put(np.asarray(rotated_dataset))

  def rotate_image(self, image, angle):
    center = tuple(np.array(image.shape[1::-1]) / 2)
    matrix = cv2.getRotationMatrix2D(center, angle, 1.0)
    result = cv2.warpAffine(image, matrix, image.shape[1::-1], flags=cv2.INTER_LINEAR)
    return result

  def prepareForFinalTrain(self):
    self.X_ALL = np.asarray(list(self.X_train) + list(self.X_valid) + list(self.X_test))
    self.Y_ALL = np.asarray(list(self.Y_train) + list(self.Y_valid) + list(self.Y_test))


In [9]:
#dataset = Dataset('Quick Draw')

In [10]:
#dataset.visualizeData(0,7)

In [33]:
class TrainableModel:
  def __init__(self, dataset, model, name = "", patience = 15):
    self.name = name
    self.dataset = dataset
    self.model = model
    self.es = EarlyStopping(monitor='val_loss', mode='min', verbose=1, patience=patience)
    self.mc = ModelCheckpoint(PATH + 'best_model.h5', monitor='val_loss', mode='min', save_best_only=True)
    self.optimizer = 'adam'
    self.compileModel()

  def evaluate(self):
    #self.loadBestModel()
    results = self.model.evaluate(self.dataset.X_test, self.dataset.Y_test, batch_size=128)
    self.test_accuracy = results[1]
    print(results)
    with open(PATH + 'History_' + str(self.name) + '_' + str(self.dataset.name) + '_' + str(self.test_accuracy) + '.json', 'w') as f:
      json.dump(self.history.history, f)
    
  def compileModel(self):
    self.model.compile(optimizer= self.optimizer, loss=tf.keras.losses.categorical_crossentropy, metrics=['accuracy'])
    
  def softmaxVectorToOneHot(self, vector):
    index = np.argmax(vector, axis=0)
    oneHot = np.zeros(31)
    oneHot[index] = 1
    return oneHot

  def submission(self):
    #self.loadBestModel()
    Y_predicted = self.model.predict(self.dataset.X_submission)
    labels = []
    for line in Y_predicted:
      labels.append(self.dataset.oneHotTransformer.inverse_transform(self.softmaxVectorToOneHot(line).reshape(1, -1))[0][0])
    df = pd.DataFrame()
    df['Category'] = labels
    df.insert(0, 'Id', range(0, len(df)))
    df.to_csv(PATH + 'Submission-' + str(self.test_accuracy) + '.csv',index=False)

  def fit(self, num_epochs = 300, batchSize = 32):
    X = self.dataset.X_train
    Y = self.dataset.Y_train
    X_valid = self.dataset.X_valid
    Y_valid = self.dataset.Y_valid
    self.history = self.model.fit(X, Y, epochs=num_epochs, batch_size = batchSize, validation_data=(X_valid, Y_valid), callbacks=[self.es, self.mc])

  def loadBestModel(self):
    self.model = load_model(PATH + 'best_model.h5')

  def fitOverAllData(self, num_epochs = 5):
    X = self.dataset.X_ALL
    Y = self.dataset.Y_ALL
    self.history = self.model.fit(X, Y, epochs=num_epochs)

  def graphFittingLoss(self):
    plt.plot(self.history.history['loss'])
    plt.plot(self.history.history['val_loss'])
    plt.title('Model loss over the training steps')
    plt.ylabel('loss')
    plt.xlabel('epoch')
    plt.legend(['train', 'test'], loc='upper left')
    plt.show()

  def graphFittingAccuracy(self):
    plt.plot(self.history.history['accuracy'])
    plt.plot(self.history.history['val_accuracy'])
    plt.title('Model accuracy over the training steps')
    plt.ylabel('accuracy')
    plt.xlabel('epoch')
    plt.legend(['train', 'test'], loc='upper left')
    plt.show()

# CNN

In [12]:
RUN = False

In [13]:
model = models.Sequential()
model.add(layers.Conv2D(64, (5, 5), activation='relu', input_shape=(100, 100, 1)))
model.add(layers.MaxPooling2D((2, 2)))
model.add(layers.Dropout(0.25))
model.add(layers.BatchNormalization())
model.add(layers.Conv2D(128, (5, 5), activation='relu'))
model.add(layers.MaxPooling2D((2, 2)))
model.add(layers.Dropout(0.25))
model.add(layers.BatchNormalization())
model.add(layers.Conv2D(256, (3, 3), activation='relu'))
model.add(layers.MaxPooling2D((2, 2)))
model.add(layers.Dropout(0.25))
model.add(layers.Flatten())
model.add(layers.Dense(256, activation='relu'))
model.add(layers.Dropout(0.5))
model.add(layers.Dense(31, activation='softmax'))
#model.summary()


In [14]:
if RUN:
  cnn = TrainableModel(dataset, model, name = "CNN")
  cnn.fit()
  cnn.evaluate()

# Big CNN

In [15]:
RUN = False

In [16]:
model = models.Sequential()
model.add(layers.Conv2D(32, (3, 3), strides=(1,1), padding="same", activation='relu', input_shape=(100, 100, 1)))
model.add(layers.BatchNormalization(momentum=0.99))
model.add(layers.Conv2D(64, (3, 3), strides=(2,2), padding="same", activation='relu'))
model.add(layers.BatchNormalization(momentum=0.99))
model.add(layers.Conv2D(32, (1, 1), strides=(1,1), padding="same", activation='relu'))
model.add(layers.BatchNormalization(momentum=0.99))
model.add(layers.Conv2D(64, (3, 3), strides=(1,1), padding="same", activation='relu'))
model.add(layers.BatchNormalization(momentum=0.99))
model.add(layers.MaxPooling2D((2, 2), strides=(2,2), padding="same"))
model.add(layers.Dropout(0.4))

model.add(layers.Conv2D(128, (3, 3), strides=(2,2), padding="same", activation='relu'))
model.add(layers.BatchNormalization(momentum=0.99))
model.add(layers.Conv2D(64, (1, 1), strides=(1,1),padding="same", activation='relu'))
model.add(layers.BatchNormalization(momentum=0.99))
model.add(layers.Conv2D(128, (3, 3), strides=(1,1), padding="same", activation='relu'))
model.add(layers.BatchNormalization(momentum=0.99))
model.add(layers.MaxPooling2D((2, 2), strides=(2,2), padding="same"))
model.add(layers.Dropout(0.4))

model.add(layers.Conv2D(64, (1, 1), strides=(1,1),padding="same", activation='relu'))
model.add(layers.BatchNormalization(momentum=0.99))
model.add(layers.Conv2D(128, (3, 3), strides=(1,1), padding="same", activation='relu'))
model.add(layers.BatchNormalization(momentum=0.99))
model.add(layers.MaxPooling2D((2, 2), strides=(2,2), padding="same"))
model.add(layers.Dropout(0.4))

model.add(layers.Conv2D(256, (3, 3), strides=(2,2),padding="same", activation='relu'))
model.add(layers.BatchNormalization(momentum=0.99))
model.add(layers.Conv2D(128, (1, 1), strides=(1,1),padding="same", activation='relu'))
model.add(layers.BatchNormalization(momentum=0.99))
model.add(layers.Conv2D(256, (3, 3), strides=(1,1),padding="same", activation='relu'))
model.add(layers.BatchNormalization(momentum=0.99))
model.add(layers.MaxPooling2D((2, 2), strides=(2,2), padding="same"))
model.add(layers.Dropout(0.4))

model.add(layers.Conv2D(128, (1, 1), strides=(1,1),padding="same", activation='relu'))
model.add(layers.BatchNormalization(momentum=0.99))
model.add(layers.Conv2D(256, (3, 3), strides=(1,1),padding="same", activation='relu'))
model.add(layers.BatchNormalization(momentum=0.99))
model.add(layers.MaxPooling2D((2, 2), strides=(2,2), padding="same"))
model.add(layers.Dropout(0.4))


model.add(layers.Conv2D(512, (3, 3), strides=(2,2),padding="same", activation='relu'))
model.add(layers.BatchNormalization(momentum=0.99))
model.add(layers.Conv2D(256, (1, 1), strides=(1,1),padding="same", activation='relu'))
model.add(layers.BatchNormalization(momentum=0.99))
model.add(layers.Conv2D(512, (3, 3), strides=(1,1),padding="same", activation='relu'))
model.add(layers.BatchNormalization(momentum=0.99))
model.add(layers.MaxPooling2D((2, 2), strides=(2,2), padding="same"))
model.add(layers.Dropout(0.4))

model.add(layers.Conv2D(256, (1, 1), strides=(1,1),padding="same", activation='relu'))
model.add(layers.BatchNormalization(momentum=0.99))
model.add(layers.Conv2D(512, (3, 3), strides=(1,1),padding="same", activation='relu'))
model.add(layers.BatchNormalization(momentum=0.99))
model.add(layers.MaxPooling2D((2, 2), strides=(2,2), padding="same"))
model.add(layers.Dropout(0.4))

model.add(layers.Conv2D(1024, (3, 3), strides=(2,2),padding="same", activation='relu'))
model.add(layers.BatchNormalization(momentum=0.99))
model.add(layers.Conv2D(512, (1, 1), strides=(1,1),padding="same", activation='relu'))
model.add(layers.BatchNormalization(momentum=0.99))
model.add(layers.Conv2D(1024, (3, 3), strides=(1,1),padding="same", activation='relu'))
model.add(layers.BatchNormalization(momentum=0.99))
model.add(layers.MaxPooling2D((2, 2), strides=(2,2), padding="same"))
model.add(layers.Dropout(0.4))

model.add(layers.Conv2D(512, (1, 1), strides=(1,1),padding="same", activation='relu'))
model.add(layers.BatchNormalization(momentum=0.99))
model.add(layers.Conv2D(1024, (3, 3), strides=(1,1),padding="same", activation='relu'))
model.add(layers.BatchNormalization(momentum=0.99))
model.add(layers.MaxPooling2D((2, 2), strides=(2,2), padding="same"))
model.add(layers.Dropout(0.4))

model.add(layers.Conv2D(512, (1, 1), strides=(1,1),padding="same", activation='relu'))
model.add(layers.BatchNormalization(momentum=0.99))
model.add(layers.Conv2D(1024, (3, 3), strides=(1,1),padding="same", activation='relu'))
model.add(layers.BatchNormalization(momentum=0.99))
model.add(layers.Conv2D(512, (1, 1), strides=(1,1),padding="same", activation='relu'))
model.add(layers.BatchNormalization(momentum=0.99))
model.add(layers.Conv2D(1024, (3, 3), strides=(1,1),padding="same", activation='relu'))
model.add(layers.BatchNormalization(momentum=0.99))
model.add(layers.Conv2D(512, (1, 1), strides=(1,1),padding="same", activation='relu'))
model.add(layers.BatchNormalization(momentum=0.99))
model.add(layers.MaxPooling2D((2, 2), strides=(2,2), padding="same"))
model.add(layers.Dropout(0.4))

model.add(layers.Conv2D(1024, (3, 3), strides=(1,1),padding="same", activation='relu'))
model.add(layers.BatchNormalization(momentum=0.99))
model.add(layers.Conv2D(256, (1, 1), strides=(1,1),padding="same", activation='relu'))
model.add(layers.BatchNormalization(momentum=0.99))
model.add(layers.MaxPooling2D((2, 2), strides=(2,2), padding="same"))
model.add(layers.Dropout(0.4))

model.add(layers.Conv2D(256, (1, 1), strides=(1,1),padding="same", activation='relu'))
model.add(layers.BatchNormalization(momentum=0.99))
model.add(layers.Conv2D(512, (3, 3), strides=(1,1),padding="same", activation='relu'))
model.add(layers.BatchNormalization(momentum=0.99))
model.add(layers.Conv2D(256, (1, 1), strides=(1,1),padding="same", activation='relu'))
model.add(layers.BatchNormalization(momentum=0.99))
model.add(layers.Conv2D(512, (3, 3), strides=(1,1),padding="same", activation='relu'))
model.add(layers.BatchNormalization(momentum=0.99))
model.add(layers.Conv2D(256, (1, 1), strides=(1,1),padding="same", activation='relu'))
model.add(layers.BatchNormalization(momentum=0.99))
model.add(layers.MaxPooling2D((2, 2), strides=(2,2), padding="same"))
model.add(layers.Dropout(0.4))

model.add(layers.Conv2D(512, (3, 3), strides=(1,1),padding="same", activation='relu'))
model.add(layers.BatchNormalization(momentum=0.99))
model.add(layers.Conv2D(256, (1, 1), strides=(1,1),padding="same", activation='relu'))
model.add(layers.BatchNormalization(momentum=0.99))
model.add(layers.MaxPooling2D((2, 2), strides=(2,2), padding="same"))
model.add(layers.Dropout(0.4))

model.add(layers.Conv2D(128, (1, 1), strides=(1,1),padding="same", activation='relu'))
model.add(layers.BatchNormalization(momentum=0.99))
model.add(layers.MaxPooling2D((2, 2), strides=(2,2), padding="same"))
model.add(layers.Dropout(0.4))

model.add(layers.Conv2D(128, (1, 1), strides=(1,1),padding="same", activation='relu'))
model.add(layers.BatchNormalization(momentum=0.99))
model.add(layers.Conv2D(256, (3, 3), strides=(1,1),padding="same", activation='relu'))
model.add(layers.BatchNormalization(momentum=0.99))
model.add(layers.Conv2D(128, (1, 1), strides=(1,1),padding="same", activation='relu'))
model.add(layers.BatchNormalization(momentum=0.99))
model.add(layers.Conv2D(256, (3, 3), strides=(1,1),padding="same", activation='relu'))
model.add(layers.BatchNormalization(momentum=0.99))
model.add(layers.Conv2D(128, (1, 1), strides=(1,1),padding="same", activation='relu'))
model.add(layers.BatchNormalization(momentum=0.99))
model.add(layers.Conv2D(256, (3, 3), strides=(1,1),padding="same", activation='relu'))
model.add(layers.BatchNormalization(momentum=0.99))
model.add(layers.MaxPooling2D((2, 2), strides=(2,2), padding="same"))
model.add(layers.Dropout(0.4))

model.add(layers.Flatten())
model.add(layers.Dense(512, activation='relu'))
model.add(layers.BatchNormalization(momentum=0.99))
model.add(layers.Dropout(0.4))
model.add(layers.Dense(512, activation='relu'))
model.add(layers.BatchNormalization(momentum=0.99))
model.add(layers.Dropout(0.4))
model.add(layers.Dense(256, activation='relu'))
model.add(layers.BatchNormalization(momentum=0.99))
model.add(layers.Dropout(0.4))
model.add(layers.Dense(128, activation='relu'))
model.add(layers.BatchNormalization(momentum=0.99))
model.add(layers.Dropout(0.4))
model.add(layers.Dense(64, activation='relu'))
model.add(layers.BatchNormalization(momentum=0.99))
model.add(layers.Dropout(0.4))
model.add(layers.Dense(31, activation='softmax'))
#model.summary()

In [17]:
if RUN:
  bigCNN = TrainableModel(dataset, model, name = "Big CNN")
  bigCNN.fit()
  bigCNN.evaluate()

# DenseNet

In [18]:
RUN = False

In [19]:
def relu(x): return layers.Activation('relu')(x)
def dropout(x, p): return layers.Dropout(p)(x) if p else x
def bn(x): return layers.BatchNormalization(axis=-1)(x)
def relu_bn(x): return relu(bn(x))

def conv(x, nf, sz, wd, p):
    x = layers.Conv2D(nf, (sz, sz), kernel_initializer='uniform', padding='same', 
                          kernel_regularizer = tf.keras.regularizers.l2(wd))(x)
    return dropout(x,p)

def conv_block(x, nf, bottleneck=False, p=None, wd=0):
    x = relu_bn(x)
    if bottleneck: x = relu_bn(conv(x, nf * 4, 1, wd, p))
    return conv(x, nf, 3, wd, p)

def dense_block(x, nb_layers, growth_rate, bottleneck=False, p=None, wd=0):
    if bottleneck: nb_layers //= 2
    for i in range(nb_layers):
        b = conv_block(x, growth_rate, bottleneck=bottleneck, p=p, wd=wd)
        x = concatenate([x,b], -1)
    return x

def transition_block(x, compression=1.0, p=None, wd=0):
    nf = int(x.get_shape().as_list()[-1] * compression)
    x = relu_bn(x)
    x = conv(x, nf, 1, wd, p)
    return layers.AveragePooling2D((2, 2), strides=(2, 2))(x)

def create_dense_net(nb_classes, img_input, depth=40, nb_block=3, 
     growth_rate=12, nb_filter=16, bottleneck=False, compression=1.0, p=None, wd=0, activation='softmax'):
    
    assert activation == 'softmax' or activation == 'sigmoid'
    assert (depth - 4) % nb_block == 0
    nb_layers_per_block = int((depth - 4) / nb_block)
    nb_layers = [nb_layers_per_block] * nb_block

    x = conv(img_input, nb_filter, 3, wd, 0)
    for i,block in enumerate(nb_layers):
        x = dense_block(x, block, growth_rate, bottleneck=bottleneck, p=p, wd=wd)
        if i != len(nb_layers)-1:
            x = transition_block(x, compression=compression, p=p, wd=wd)

    x = relu_bn(x)
    x = layers.GlobalAveragePooling2D()(x)
    return layers.Dense(nb_classes, activation=activation, kernel_regularizer=regularizers.l2(wd))(x)

In [20]:
input = layers.Input(shape=(100, 100, 1))
output = create_dense_net(31, input, depth = 16)
model = models.Model(input, output)
#model.summary()

In [21]:
if RUN:
  denseNet = TrainableModel(dataset, model, name = "DenseNet")
  denseNet.optimizer = tf.keras.optimizers.SGD(0.1, 0.9, nesterov=True)
  denseNet.compileModel()
  denseNet.fit()
  denseNet.evaluate()
  denseNet.submission()

# Densenet on preprocessed and augmented data

In [22]:
RUN = True

In [24]:
angles = np.arange(-45, 45, 30)
alphas = [20, 10, 5]
sigmas = [4, 4, 2]

In [25]:
dataset_Pre_Aug = Dataset('Data_flip_angles-45To45_5deform', True, True, True, alpha = alphas, sigma = sigmas, rotate_augment = True, angles = angles, flipAND_ = True)

Augmentation of the dataset by deformation
***
 preparing for join
***

Augmentation of the dataset by rotation
***
 preparing for join
***

In [26]:
#dataset_Pre_Aug.visualizeData(345590,345599)

In [27]:
#dataset_Pre_Aug.oneHotTransformer.inverse_transform( dataset_Pre_Aug.Y_train[403190].reshape(1, -1))

In [28]:
len(dataset_Pre_Aug.X_train)

100800

In [29]:
len(dataset_Pre_Aug.X_test)

1000

In [30]:
len(dataset_Pre_Aug.X_valid)

1800

In [34]:
input = layers.Input(shape=(32, 32, 1))
output = create_dense_net(31, input, depth=124, nb_filter=12, growth_rate=36, compression=0.2, bottleneck=True, p=0.2, wd=1e-4)
model = models.Model(input, output)
#model.summary()

In [35]:
if RUN:
  denseNet = TrainableModel(dataset_Pre_Aug, model, name = "DenseNet40layersBottleneckCompressionDropout0.2", patience = 35)
  denseNet.optimizer = tf.keras.optimizers.SGD(0.5, 0.9, nesterov=True)
  
  denseNet.compileModel()
  denseNet.fit(num_epochs = 5, batchSize = 32)

  denseNet.optimizer = tf.keras.optimizers.SGD(0.01, 0.9, nesterov=True)
  denseNet.compileModel()
  denseNet.fit(num_epochs = 40, batchSize = 32)

  denseNet.evaluate()
  denseNet.submission()

Epoch 1/5
3150/3150 [==============================] - 459s 146ms/step - loss: 2.6921 - accuracy: 0.4139 - val_loss: 2.7254 - val_accuracy: 0.4272
Epoch 2/5
3150/3150 [==============================] - 450s 143ms/step - loss: 1.9132 - accuracy: 0.6317 - val_loss: 2.1995 - val_accuracy: 0.5806
Epoch 3/5
3150/3150 [==============================] - 451s 143ms/step - loss: 1.8382 - accuracy: 0.6784 - val_loss: 2.0711 - val_accuracy: 0.6372
Epoch 4/5
3150/3150 [==============================] - 450s 143ms/step - loss: 1.8009 - accuracy: 0.6981 - val_loss: 1.9759 - val_accuracy: 0.6706
Epoch 5/5
3150/3150 [==============================] - 456s 145ms/step - loss: 1.7903 - accuracy: 0.7071 - val_loss: 1.9115 - val_accuracy: 0.6750
Epoch 1/40
3150/3150 [==============================] - 453s 144ms/step - loss: 1.3855 - accuracy: 0.8069 - val_loss: 1.3539 - val_accuracy: 0.8178
Epoch 2/40
3150/3150 [==============================] - 450s 143ms/step - loss: 1.2091 - accuracy: 0.8354 - val_loss:

KeyboardInterrupt: ignored

In [ ]:
#denseNet.fitOverAllData(num_epochs = 1)

In [32]:
denseNet.evaluate()

57/57 [==============================] - 1s 17ms/step - loss: 1.3253 - accuracy: 0.8172
[1.3253268003463745, 0.8172222375869751]


NameError: ignored